<h1>Extract participant video from zoom recording</h1>

In [5]:
import subprocess

# Cut the video
input_video = 'zoom_0.mp4'
start_time = '00:07:19'  
end_time = '01:22:35'    
output_cut = 'ID98_shorter.mp4'

cut_command = f"ffmpeg -i {input_video} -ss {start_time} -to {end_time} -c copy {output_cut}"
result = subprocess.run(cut_command, shell=True)

# # Extract Face Video
# output_cropped = 'output_cropped.mp4'

# crop_command = f"ffmpeg -i {output_cut} -analyzeduration 10M -probesize 10M -c:v libx264 -vf 'crop=640:480:10:10' {output_cropped}"

# # Run the command and capture output
# result = subprocess.run(crop_command, shell=True, capture_output=True, text=True)

# Print the output
print("FFmpeg Output:", result.stdout)

# Check for errors
if result.returncode != 0:
    print("Error:", result.stderr)


FFmpeg Output: None


In [22]:
import cv2
import time

# Input and output video filenames
input_video_filename = 'ID98_shorter.mp4'
output_cropped_filename = 'ID98_cropped.mp4'

# Open the video file
cap = cv2.VideoCapture(input_video_filename)

if not cap.isOpened():
    print(f"Error: Could not open video '{input_video_filename}'.")
    exit()

# Initialize retry mechanism variables
max_attempts = 5
attempts = 0
frame_read_successfully = False

while attempts < max_attempts and not frame_read_successfully:
    ret, frame = cap.read()
    if ret:
        frame_read_successfully = True
    else:
        print(f"Attempt {attempts + 1} of {max_attempts}: Failed to read first frame. Retrying...")
        time.sleep(1)  # Wait for 1 second before retrying
        attempts += 1

if not frame_read_successfully:
    print("Failed to read the first frame after multiple attempts. Exiting...")
    cap.release()
    exit()

# Reset the video capture to the start if needed
cap.set(cv2.CAP_PROP_POS_FRAMES, 0)

# Get video properties
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Set the desired crop region (adjust these values as needed)
x, y, w, h = width - 200, 130, 200, 130

# Create a VideoWriter object to write the cropped video
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # You can change the codec if needed
out = cv2.VideoWriter(output_cropped_filename, fourcc, 30, (w, h))

# Read and write frames
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Crop the frame
    cropped_frame = frame[y:y+h, x:x+w]

    # Write the cropped frame to the output video
    out.write(cropped_frame)

# Release the video capture and writer objects
cap.release()
out.release()

# Destroy any OpenCV windows
cv2.destroyAllWindows()


Attempt 1 of 5: Failed to read first frame. Retrying...
